# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [86]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [87]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [88]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [89]:
songs = pd.read_csv('event_datafile_new.csv')

In [90]:
songs.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [91]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [92]:
songs.head(10)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38
5,Carrie Underwood,Gianna,F,2,Jones,195.70893,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Look At Me,38
6,No Te Va Gustar,Kaylee,F,0,Summers,246.43873,free,"Phoenix-Mesa-Scottsdale, AZ",833,Ya EntendÃÂ­,8
7,The Velvet Underground / Nico,Kaylee,F,1,Summers,360.09751,free,"Phoenix-Mesa-Scottsdale, AZ",833,All Tomorrow's Parties,8
8,Snoop Dogg / Jamie Foxx,Kaylee,F,2,Summers,179.35628,free,"Phoenix-Mesa-Scottsdale, AZ",833,Psst!,8
9,Dixie Chicks,Kaylee,F,3,Summers,201.01179,free,"Phoenix-Mesa-Scottsdale, AZ",833,There's Your Trouble,8


In [93]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [94]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [95]:
# CREATE KEYSPACE 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS project 
                    WITH REPLICATION =
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

except Exception as e:
    print(e)

#### Set Keyspace

In [96]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'





In [97]:
#We create a table for each query. In this one we chose sessionID & itemInSession casue they have to be unique.

query = "CREATE TABLE IF NOT EXISTS music_history"
query = query + "(artist text, itemInSession int , length float, sessionId int, song text, \
                  PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [98]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        #In every insert part we are going to INSERT the columns we put to create the tables and also add "session.execute", with each order they have in the graphic list they showed us.
        Insert_query = "INSERT INTO music_history(artist, itemInSession, length, sessionId, song)"
        Insert_query = Insert_query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(Insert_query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [99]:
#The SELECT part we do it the same as the select, just changing the data in order of the query we have to accomplish. 
query = "SELECT artist, song, length FROM music_history WHERE sessionId = 338 AND itemInSession = 4;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [100]:
#In the second query we have to sort in a descendant way 1 column of our table. Thats why i added the "With clustering order by".

query1 = "CREATE TABLE IF NOT EXISTS artist_library"
query1 = query1 + "(artist text, firstName text, itemInSession int, lastName text, sessionId int, song text, userId int,  \
                    PRIMARY KEY ((userId, sessionId), itemInSession)) \
                    WITH CLUSTERING ORDER BY (itemInSession DESC);"

try:
    session.execute(query1)
except Exception as e:
    print(e)
    


In [101]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        Insert_Query_1 = "INSERT INTO artist_library(artist, firstName, itemInSession, lastName, sessionId, song, userId)"
        Insert_Query_1 = Insert_Query_1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(Insert_Query_1, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))

In [102]:
#In this case we chose artist, song, firstName and lastName because these are the columns the second query wanted. 
query1 = "SELECT artist, song, firstName, lastName FROM artist_library WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)                    

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [103]:
#For this last query we are using as composite key the columns song title & userID, because multiple users can listen the same song.

query2 = "CREATE TABLE IF NOT EXISTS listeners_registry"
query2 = query2 + "(firstName text, lastName text,song text, userId int, \
                    PRIMARY KEY(song, userId))"
                     
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        Insert_Query_2 = "INSERT INTO listeners_registry(firstName, lastName, song, userId)"
        Insert_Query_2 = Insert_Query_2 + " VALUES ( %s, %s, %s, %s)"
        session.execute(Insert_Query_2, ( line[1], line[4], line[9], int(line[10])))

In [104]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query2 = "SELECT firstName, lastName FROM listeners_registry WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.firstname, row.lastname)   
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [85]:
#At the end always drop the table so the next time you create the same table, you are not creating multiple tables so the data gives no error.
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table listeners_registry"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()